# **Bioinformatics Project - Computational Drug Discovery [Part 3] Descriptor Calculation and Dataset Preparation**

**MOUNSIF EL ATOUCH**

In this Jupyter notebook, we will be building a machine learning model using the ChEMBL bioactivity data.

In **Part 3**, we will be calculating molecular descriptors that are essentially quantitative description of the compounds in the dataset. Finally, we will be preparing this into a dataset for subsequent model building in Part 4.

---

## **Download PaDEL-Descriptor**

In [1]:
! wget https://github.com/dataprofessor/bioinformatics/raw/master/padel.zip
! wget https://github.com/dataprofessor/bioinformatics/raw/master/padel.sh

--2023-05-28 21:52:40--  https://github.com/dataprofessor/bioinformatics/raw/master/padel.zip
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dataprofessor/bioinformatics/master/padel.zip [following]
--2023-05-28 21:52:40--  https://raw.githubusercontent.com/dataprofessor/bioinformatics/master/padel.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25768637 (25M) [application/zip]
Saving to: ‘padel.zip’

padel.zip           100%[===================>]  24.57M  --.-KB/s    in 0.1s    

2023-05-28 21:52:41 (238 MB/s) - ‘padel.zip’ saved [25768637/25768637]

--2023-05-28 21:52:41--  https://gi

In [2]:
! unzip padel.zip

Archive:  padel.zip
   creating: PaDEL-Descriptor/
  inflating: __MACOSX/._PaDEL-Descriptor  
  inflating: PaDEL-Descriptor/MACCSFingerprinter.xml  
  inflating: __MACOSX/PaDEL-Descriptor/._MACCSFingerprinter.xml  
  inflating: PaDEL-Descriptor/AtomPairs2DFingerprinter.xml  
  inflating: __MACOSX/PaDEL-Descriptor/._AtomPairs2DFingerprinter.xml  
  inflating: PaDEL-Descriptor/EStateFingerprinter.xml  
  inflating: __MACOSX/PaDEL-Descriptor/._EStateFingerprinter.xml  
  inflating: PaDEL-Descriptor/Fingerprinter.xml  
  inflating: __MACOSX/PaDEL-Descriptor/._Fingerprinter.xml  
  inflating: PaDEL-Descriptor/.DS_Store  
  inflating: __MACOSX/PaDEL-Descriptor/._.DS_Store  
   creating: PaDEL-Descriptor/license/
  inflating: __MACOSX/PaDEL-Descriptor/._license  
  inflating: PaDEL-Descriptor/KlekotaRothFingerprintCount.xml  
  inflating: __MACOSX/PaDEL-Descriptor/._KlekotaRothFingerprintCount.xml  
  inflating: PaDEL-Descriptor/config  
  inflating: __MACOSX/PaDEL-Descriptor/._config  
  inf

## **Installing librairies**

In [3]:
! pip install rdkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 36.7 MB/s eta 0:00:00


## **Importing libraires**

In [4]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

## **Load bioactivity data**

Download the curated ChEMBL bioactivity data that has been pre-processed from Parts 1 and 2 of this Bioinformatics Project series. Here we will be using the **bioactivity_data_curated.csv** file that essentially contain the pIC50 values that we will be using for building a classification model.

In [23]:
df = pd.read_csv('bioactivity_data_curated.csv')

In [26]:
selection = ['canonical_smiles','chembl_id']
df_selection = df[selection]
df_selection.to_csv('molecule.smi', sep='\t', index=False, header=False)

In [27]:
! cat molecule.smi | head -5

O=c1oc2ccccc2c(O)c1Cc1c(O)c2ccccc2oc1=O	CHEMBL1466
O=c1oc2ccccc2c(O)c1C(c1ccc[nH]1)c1c(O)c2ccccc2oc1=O	CHEMBL260998
CCOc1cc(C(c2c(O)c3ccccc3oc2=O)c2c(O)c3ccccc3oc2=O)ccc1O	CHEMBL260997
O=c1oc2ccccc2c(O)c1C(c1c(O)c2ccccc2oc1=O)c1c[nH]c2ccccc12	CHEMBL258733
O=c1c(C(c2ccc([N+](=O)[O-])cc2)c2c(O)oc3ccccc3c2=O)c(O)oc2ccccc12	CHEMBL81935


In [28]:
! cat molecule.smi | wc -l

648


## **Calculate fingerprint descriptors**


### **Calculate PaDEL descriptors**

In [29]:
! cat padel.sh

java -Xms1G -Xmx1G -Djava.awt.headless=true -jar ./PaDEL-Descriptor/PaDEL-Descriptor.jar -removesalt -standardizenitro -fingerprints -descriptortypes ./PaDEL-Descriptor/PubchemFingerprinter.xml -dir ./ -file descriptors_output.csv


In [30]:
%%time
! bash padel.sh

Processing CHEMBL260998 in molecule.smi (2/648). 
Processing CHEMBL1466 in molecule.smi (1/648). 
Processing CHEMBL258733 in molecule.smi (4/648). Average speed: 2.21 s/mol.
Processing CHEMBL260997 in molecule.smi (3/648). Average speed: 3.56 s/mol.
Processing CHEMBL81935 in molecule.smi (5/648). Average speed: 2.13 s/mol.
Processing CHEMBL409739 in molecule.smi (6/648). Average speed: 1.72 s/mol.
Processing CHEMBL430228 in molecule.smi (7/648). Average speed: 1.65 s/mol.
Processing CHEMBL409439 in molecule.smi (8/648). Average speed: 1.44 s/mol.
Processing CHEMBL259477 in molecule.smi (9/648). Average speed: 1.41 s/mol.
Processing CHEMBL259153 in molecule.smi (10/648). Average speed: 1.27 s/mol.
Processing CHEMBL259154 in molecule.smi (11/648). Average speed: 1.23 s/mol.
Processing CHEMBL409740 in molecule.smi (12/648). Average speed: 1.14 s/mol.
Processing CHEMBL261558 in molecule.smi (13/648). Average speed: 1.16 s/mol.
Processing CHEMBL261789 in molecule.smi (14/648). Average speed

## **Preparing the X and Y Data Matrices**

### **X data matrix**

In [31]:
df_X = pd.read_csv('descriptors_output.csv')

In [32]:
df_X = df_X.drop(columns=['Name'])

In [33]:
df_X

,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,PubchemFP7,PubchemFP8,PubchemFP9,...,PubchemFP871,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880
0,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
643,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
644,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
645,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
646,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


## **Y variable: Transformation into a binary variable**
The bioactivity data is in the pIC50 unit. Compounds having values of >= 6 will be considered to be **active = `1`** while those > 6 will be considered to be **inactive = `0`**.

In [37]:
def get_class(y, thres=6):
  labels = {0: '0', 1: '1'}
  return labels[1] if y >= thres else labels[0]

In [38]:
df_Y = df['pIC50']

In [39]:
df_Y

0      4.820000
1      4.330000
2      4.040000
3      4.210000
4      4.150000
         ...   
643    3.966576
644    4.450997
645    4.070581
646    4.696804
647    3.100179
Name: pIC50, Length: 648, dtype: float64

In [40]:
df_Y = df_Y.apply(get_class)

## **Combining X and Y variable**

In [43]:
data = pd.concat([df_X, df_Y], axis=1)

In [45]:
data = data.rename(columns={'pIC50': 'Activity'})

In [46]:
data

,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,PubchemFP7,PubchemFP8,PubchemFP9,...,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880,Activity
0,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
643,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
644,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
645,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
646,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [47]:
data.to_csv('bioactivity_data_pubchem_fp.csv', index=False)

In [48]:
! zip lab3.zip *

  adding: bioactivity_data_curated.csv (deflated 77%)
  adding: bioactivity_data_pubchem_fp.csv (deflated 97%)
  adding: descriptors_output.csv (deflated 96%)
  adding: __MACOSX/ (stored 0%)
  adding: molecule.smi (deflated 82%)
  adding: PaDEL-Descriptor/ (stored 0%)
  adding: padel.sh (deflated 36%)
  adding: padel.zip (stored 0%)
  adding: sample_data/ (stored 0%)
